In [1]:
import torch
import torch as F
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision import datasets
import matplotlib.pyplot as plt
import numpy as np
import cv2

import os
os.chdir("../models")
from common_utils import *
from gradmap_utils import *

# set seed
set_seed(42)

In [2]:
transform2 = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((100, 100), antialias=True),
])


# load data
test_dataset = CustomFlowers102(root='../data', split='train', download=True, transform=transform2)

# define dataloader to load single image
dataloader = DataLoader(dataset=test_dataset, shuffle=False, batch_size=1)

In [3]:
# specify model path
model_path = 'saved_models/FinalModel/cuda/best_model.pt'
# determine device type
if torch.cuda.is_available(): # nvidia gpu
    device = torch.device("cuda")
elif torch.backends.mps.is_available(): # apple gpu
    device = torch.device("mps")
else:
    device = torch.device("cpu")


# load model
model = cnn_grad_cam(model_path=model_path, device=device)
model.eval()

# create directory to save grad maps
if not os.path.exists('../grad_maps'):
    os.makedirs('../grad_maps')

path = '../grad_maps'

# iterate over images in dataloader
for i, (img, label, img_path) in enumerate(dataloader):
    # get heatmap
    heatmap = get_heatmap(model, img)
    img_path = img_path[0]
    # resize image to larger size
    img = cv2.imread(img_path)
    # get filename of img_path
    filename = os.path.basename(img_path)

    # path to save grad map
    save_path = os.path.join(path, filename)

    # save grad_map
    save_grad_map(img, heatmap, save_path)
    
    if i == 9:
        break